In [1]:
import numpy as np 
import pandas as pd
import io
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/aircraft-accidents-failures-hijacks-dataset/Aircraft_Incident_Dataset.csv
/kaggle/input/airline-dataset/travel.sqlite


In [2]:
%load_ext sql

In [3]:
%%sql 
sqlite:////kaggle/input/airline-dataset/travel.sqlite

get 15 flights that have the highest total revenue

In [4]:

%%sql
SELECT
    f.flight_id,
    f.flight_no,
    a1.city->>'en' AS departure_city,
    a2.city->>'en' AS arrival_city,
    SUM(tf.amount) AS total_revenue
FROM
    flights f
JOIN
    ticket_flights tf ON f.flight_id = tf.flight_id
JOIN
    airports_data a1 ON f.departure_airport = a1.airport_code
JOIN
    airports_data a2 ON f.arrival_airport = a2.airport_code
GROUP BY
    f.flight_id, f.flight_no, a1.city, a2.city
ORDER BY
    total_revenue DESC
LIMIT 15;

 * sqlite:////kaggle/input/airline-dataset/travel.sqlite
Done.


flight_id,flight_no,departure_city,arrival_city,total_revenue
2354,PG0208,Moscow,Khabarovsk,17146600
26212,PG0209,Khabarovsk,Moscow,17023600
2364,PG0208,Moscow,Khabarovsk,16962100
2330,PG0208,Moscow,Khabarovsk,16900600
2329,PG0208,Moscow,Khabarovsk,16894400
2341,PG0208,Moscow,Khabarovsk,16839100
2344,PG0208,Moscow,Khabarovsk,16777600
2369,PG0208,Moscow,Khabarovsk,16777600
2325,PG0208,Moscow,Khabarovsk,16593100
2332,PG0208,Moscow,Khabarovsk,16593100


Get top 500 routes by total revenue

In [5]:
%%sql result_set <<

SELECT
    f.flight_id,
    f.flight_no,
    a1.city->>'en' AS departure_city,
    a2.city->>'en' AS arrival_city,
    SUM(tf.amount) AS total_revenue
FROM
    flights f
JOIN
    ticket_flights tf ON f.flight_id = tf.flight_id
JOIN
    airports_data a1 ON f.departure_airport = a1.airport_code
JOIN
    airports_data a2 ON f.arrival_airport = a2.airport_code
GROUP BY
    f.flight_id, f.flight_no, a1.city, a2.city
ORDER BY
    total_revenue DESC
LIMIT 500

 * sqlite:////kaggle/input/airline-dataset/travel.sqlite
Done.
Returning data to local variable result_set


In [6]:
df = result_set.DataFrame()
df.to_csv('topsell.csv', index=False)

get top 100 flights with highest total revenue grouped by fare type

In [7]:
%%sql result_set <<
SELECT
    f.flight_id,
    f.flight_no,
    tf.fare_conditions,
    SUM(tf.amount) AS sum_revenue
FROM
    flights f
JOIN
    ticket_flights tf ON f.flight_id = tf.flight_id
GROUP BY
    f.flight_no, tf.fare_conditions
ORDER BY
    sum_revenue DESC
LIMIT 100;

 * sqlite:////kaggle/input/airline-dataset/travel.sqlite
Done.
Returning data to local variable result_set


In [8]:
df = result_set.DataFrame()
df.to_csv('faretype.csv', index=False)

get average revenue for each aircraft model

In [9]:
%%sql result_set <<
SELECT
    aircd.aircraft_code,
    aircd.model->>'en' AS model_name,
    AVG(tf.amount) AS avg_revenue
FROM
    aircrafts_data aircd
JOIN
    flights f ON aircd.aircraft_code = f.aircraft_code
JOIN
    ticket_flights tf ON f.flight_id = tf.flight_id
GROUP BY
    aircd.aircraft_code, aircd.model;

 * sqlite:////kaggle/input/airline-dataset/travel.sqlite
Done.
Returning data to local variable result_set


In [10]:
df = result_set.DataFrame()
df.to_csv('model_sales.csv', index=False)

In [11]:
%%sql
SELECT
    SUM(tf.amount) AS total_revenue
FROM
    aircrafts_data aircd
JOIN
    flights f ON aircd.aircraft_code = f.aircraft_code
JOIN
    ticket_flights tf ON f.flight_id = tf.flight_id;

 * sqlite:////kaggle/input/airline-dataset/travel.sqlite
Done.


total_revenue
20766980900


In [12]:
%%sql
SELECT
    aircd.aircraft_code,
    aircd.model->>'en' AS model_name,
    AVG(tf.amount) AS avg_revenue
FROM
    aircrafts_data aircd
JOIN
    flights f ON aircd.aircraft_code = f.aircraft_code
JOIN
    ticket_flights tf ON f.flight_id = tf.flight_id
GROUP BY
    aircd.aircraft_code, aircd.model;

 * sqlite:////kaggle/input/airline-dataset/travel.sqlite
Done.


aircraft_code,model_name,avg_revenue
319,Airbus A319-100,51201.693375967305
321,Airbus A321-200,15291.509301869708
733,Boeing 737-300,16568.164502566724
763,Boeing 767-300,35033.55747190921
773,Boeing 777-300,23765.760929794425
CN1,Cessna 208 Caravan,6568.552344601963
CR2,Bombardier CRJ-200,13207.66110230346
SU9,Sukhoi Superjet-100,13985.541895225022
